# Negative samples for the dataset

In [1]:
import pandas as pd
import numpy as np
import pydicom
import png
import math
from PIL import Image
from pathlib import Path
import boto3
import matplotlib.pyplot as plt

data_path = '/home/szelesteya/projects/EMBED_Open_Data/'
tables_path = data_path + 'tables/'
image_root_path = '/media/szelesteya/F824D4D024D492CC/EMBED-images/'
image_dcm_path = image_root_path + 'dicom-negative/'
image_png_path = image_root_path + 'negative-full'

In [2]:
cli_df = pd.read_csv(tables_path + 'EMBED_OpenData_clinical.csv', low_memory=False)

# Only keeping result BIRADS-1 and BIRADS-2 screenings
neg_cli_df = cli_df[(cli_df.asses.isin(['N'])) & 
                    cli_df.desc.str.contains('screen', case=False)][['Unnamed: 0',
                                                                     'empi_anon',
                                                                     'acc_anon',
                                                                     'side',
                                                                     'calcfind',
                                                                     'calcdistri',
                                                                     'otherfind',
                                                                     'numfind',
                                                                     'path_severity',
                                                                     'age_at_study',
                                                                     'ETHNICITY_DESC',
                                                                     'study_date_anon',
                                                                     'asses']]

In [3]:
# Merging clinical information with medical ones
meta_df = (pd.read_csv(tables_path + 'EMBED_OpenData_metadata_reduced.csv', low_memory=False))
meta_red_df = meta_df[(meta_df['spot_mag'] != 1) &
                      (meta_df['FinalImageType'] == '2D') &
                      (meta_df['ViewPosition'] == 'CC')][[  'empi_anon',
                                                            'acc_anon',
                                                            'ImageLateralityFinal',
                                                            'anon_dicom_path',
                                                            'ViewPosition',
                                                            'ROI_coords']]

# Rename columns to prepare for merge
meta_red_ren_df = meta_red_df.rename(columns={'ImageLateralityFinal':'side'})

In [4]:
neg_full_df = neg_cli_df.merge(meta_red_ren_df, on=['empi_anon','acc_anon','side'])

# Keeping relevant columns
neg_empi_df = neg_full_df[[ 'empi_anon',
                           'acc_anon',
                           'anon_dicom_path',
                           'side',
                           'asses',
                           'age_at_study',
                           'calcfind',
                           'calcdistri',
                           'otherfind',
                           'numfind',
                           'ROI_coords',
                           'ETHNICITY_DESC',
                           'study_date_anon']]

# Rename columns to be more consistent
neg_empi_df = neg_empi_df.rename(columns={'ETHNICITY_DESC':'eth_desc',
                                          'calcfind':'calc_find',
                                          'calcdistri':'calc_distrib',
                                          'otherfind':'other_find',
                                          'numfind':'num_find'})

# Generate paths for png extraction
neg_empi_df.loc[:, 'relative_dcm_path'] = neg_empi_df['anon_dicom_path'].apply(lambda x: '/'.join(x.split('/')[5:]))

In [5]:
# Ordering negative data by study date
neg_empi_ord_df = neg_empi_df.sort_values(by=['study_date_anon'], ascending=False)

# Choosing unique patients
neg_empi_fin_df = neg_empi_ord_df.drop_duplicates(subset=['empi_anon','acc_anon','side'], keep='first')[['empi_anon',
                                                                                       'acc_anon',
                                                                                       'side',
                                                                                       'asses',
                                                                                       'age_at_study',
                                                                                       'relative_dcm_path',
                                                                                       'calc_find',
                                                                                       'calc_distrib',
                                                                                       'other_find',
                                                                                       'num_find',
                                                                                       'ROI_coords',
                                                                                       'eth_desc',
                                                                                       'study_date_anon']]

In [28]:
# neg_empi_fin_df[(neg_empi_fin_df['ROI_coords'] == '()') & neg_empi_fin_df['num_find'] == 1]
neg_empi_find_unique_df = neg_empi_find_unique_df.drop_duplicates(subset=['empi_anon','acc_anon','side'])

NameError: name 'neg_empi_find_unique_df' is not defined

NameError: name 'neg_empi_find_unique_df' is not defined

In [24]:
with open(data_path + 'negative_empirical.csv', 'w') as f:
    neg_empi_fin_df.to_csv(f, index=False)

In [10]:
with open(data_path + 'negative_path.csv', 'w') as f:
    neg_empi_fin_df['relative_dcm_path'].to_csv(f, index=False)

In [23]:
%%bash -s "$image_dcm_path" "{data_path}negative_path.csv"

# Pulling dicom files with AWS CLI (Python API didn't work)
dcm_dest_path="$1"
dcm_paths="$2"
ind=$((1))

tail -n +2 $dcm_paths | head -n 300 | tail -n 100 | while IFS= read -r line; do
    relative_path=$(echo "$line" | awk -v OFS='/' '{$1=$1; print}')
    dcm_name=$(echo "$relative_path" | cut -d '/' -f 3-)
               
    file="${dcm_dest_path}$relative_path"
    dir=$(dirname "${dcm_dest_path}$dcm_name")
    echo "$ind / 1000"
    # echo "$file"
    # echo "${dcm_dest_path}$dcm_name"
    # echo "$dir"
    if [ -f "${dcm_dest_path}$dcm_name" ]; then
        echo "Moving file $file"
        mkdir -p $(dirname "$file") & mv "${dcm_dest_path}$dcm_name" "${dcm_dest_path}$relative_path"
        rmdir $dir
    else        
        if [ -f "$file" ]; then
            echo "File already present"
            rmdir $dir
        else
            echo "Pulling file $file"
            aws s3 cp "s3://embed-dataset-open/images/$relative_path" "${dcm_dest_path}/$dcm_name" --profile my-dev-profile
        fi
    fi

    ind=$((ind+1))
    clear
done

1 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.848.113972.3.63.1.59336055.20190518.1102855/1.2.842.113686.2750824662.1557987688.4775.34783': No such file or directory


2 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.848.113976.3.63.1.59845659.20190808.1141614/1.2.841.113688.2750828055.1564639049.2066.719408': Directory not empty


3 / 1000
File already present


...

rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.841.113974.3.57.1.60070734.20190912.1123827/1.2.841.113688.2750824972.1567578527.5088.31741': Directory not empty


98 / 1000
Moving file /media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/cohort_1/26360215/1.2.848.113977.3.58.1.58162653.20181119.1082051/1.2.847.113683.2750824657.1542349353.4287.103155/1.2.826.0.1.3680043.8.498.12988159840455631474285742490018401254.dcm
99 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.845.113979.3.58.1.60137737.20190916.1074843/1.2.849.113681.2750824487.1568615522.4710.11063': No such file or directory


100 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.840.113971.3.61.1.58474120.20190108.1130021/1.2.840.113689.2750824662.1546842756.4655.311817': No such file or directory


In [9]:
# Rescale the intensity of the image to get heterogene images with the bit depth of 14
def rescale_to_8bit(image_array):
    upper_percentile = np.percentile(image_array.flatten(), 98) # original_max = np.max(image_array)
    lower_percentile = np.percentile(image_array.flatten(), 2) # original_min = np.min(image_array)
    # max_on_14bit = 16383
    max = 255
    rescaled_array = (image_array - lower_percentile) / (upper_percentile - lower_percentile)
    rescaled_array[rescaled_array < 0] = 0
    rescaled_array[rescaled_array > 1] = 1
    # rescaled_array = np.round((image_array - original_min) / (original_max - original_min) * max_on_14bit).astype(int)
    return np.round(rescaled_array * 255).astype(np.uint8)

# Save DICOM pixel array as PNG
def save_dcm_image_as_png(image, png_filename, bitdepth=8):
    with open(png_filename, 'wb') as f:
        rescaled = rescale_to_8bit(image)
        writer = png.Writer(height=rescaled.shape[0], 
                            width=rescaled.shape[1], 
                            bitdepth=bitdepth, 
                            greyscale=True)
        writer.write(f, rescaled.tolist())

def generate_png_path(empi_anon, png_dir):
    # Get new file name
    new_fn = f"{empi_anon}_neg_conv.png"
    return f'{png_dir}/{new_fn}'

# Convert list of DICOMs to PNGs
def process_dcm_list(dcm_list, png_list):    
    for i, dcm_path in enumerate(dcm_list):    
        if not Path(png_list[i]).exists():
            print(f"Processing DICOM #{i}...")
            
            # Load DICOM
            dcm = pydicom.dcmread(dcm_path)
            img = dcm.pixel_array
            
            # Save PNG            
            save_dcm_image_as_png(img, png_list[i])

def extract_images(data_file_name, dcm_dir, png_dir):
    # Provide a list of DICOM paths and a target directory
    dcm_list = []
    df = pd.read_csv(data_file_name)
    
    for index, row in df.iterrows():
        path = dcm_dir  + row['relative_dcm_path']
        if Path(path).exists():
            dcm_list.append(path)
        
    # Insert png path
    df.loc[:,'png_path'] = df['empi_anon'].apply(lambda x: generate_png_path(x, png_dir))

    # Convert DICOMs
    process_dcm_list(dcm_list[0:300], df['png_path'])

    return df

In [10]:
neg_img_emp = extract_images(data_path + 'negative_empirical.csv', image_dcm_path, image_png_path)
# negative['intensity'] = calculate_intensity_of_png(negative)

with open(data_path + 'negative_empirical_png.csv', 'w') as f:
    (neg_img_emp)[['empi_anon',
                'acc_anon',
                'side',
                'age_at_study',
                'relative_dcm_path',
                'calc_find',
                'calc_distrib',
                'other_find',
                'num_find',
                'eth_desc',
                'study_date_anon',
                'png_path']].to_csv(f, index=False)

Processing DICOM #64...
Processing DICOM #65...
Processing DICOM #66...
Processing DICOM #67...
Processing DICOM #68...
Processing DICOM #69...
Processing DICOM #70...
Processing DICOM #71...
Processing DICOM #72...
Processing DICOM #73...
Processing DICOM #74...
Processing DICOM #75...
Processing DICOM #76...
Processing DICOM #77...
Processing DICOM #78...
Processing DICOM #79...
Processing DICOM #80...
Processing DICOM #81...
Processing DICOM #82...
Processing DICOM #83...
Processing DICOM #84...
Processing DICOM #85...
Processing DICOM #86...
Processing DICOM #87...
Processing DICOM #88...
Processing DICOM #89...
Processing DICOM #90...
Processing DICOM #91...
Processing DICOM #92...
Processing DICOM #93...
Processing DICOM #94...
Processing DICOM #95...
Processing DICOM #96...
Processing DICOM #97...
Processing DICOM #98...
Processing DICOM #99...
Processing DICOM #100...
Processing DICOM #101...
Processing DICOM #102...
Processing DICOM #103...
Processing DICOM #104...
Processing 